In [168]:
import pandas as pd
import math
from IPython.display import display, Latex

#FUNCTIONS 

# covert from degrees to radians
def convert_to_radians(deg):
    return deg * math.pi/180

# Func to format to sig fig
def format_sig_fig(x, sig):
    if isinstance(x, (int, float)):
        if x == 0:  # Check if the value is exactly 0
            return "0"
        return f"{x:.{sig}e}" if abs(x) >= 1e6 or abs(x) < 1e-3 else f"{x:.{sig}g}"
    return x

def to_latex_bold_headers(df, caption, label):
    # Build the header row with bold column names
    bold_header = " & ".join([f"\\bfseries {col}" for col in df.columns]) + " \\\\\\midrule"
    
    # Generate the base LaTeX table without the default column headers
    latex_table = df.to_latex(
        index=False,
        header=False,  # Disable default headers to avoid duplication
        escape=False,  # Allow math symbols like ^
        caption=caption,
        label=label,
        bold_rows=False  # Avoid bolding row names
    )
    
    # Replace the top rule and add the custom bold header
    latex_table = latex_table.replace("\\toprule", f"\\toprule\n{bold_header}")
    return latex_table


In [ ]:
#DEFINE CONSTANTS
g = 9.81 #m/s

truss_num = 7
beams_num = 14

# Pool Dimensions
pool_l = 16.1                          # length of pool in meters
pool_w = 16.1                          # width of pool in meters
pool_d = 0.24                          # depth of pool in meters

# Glass Modular Dimensions - Laminated Tempered Glass
glass_slab_l = 2.3                         # length of rectangular glass slab in meters
glass_slab_w = 1.15                    # width of rectangular glass slab in meters  
glass_slab_t = 0.06                    # total thickness of glass slab, 60mm

t_PVB = 0.0152                         # thickness of PVB coating layer between two layers of tempered glass, 1.52mm
t_tempered = 0.0294                    # thickness of one layer of tempered glass, 29.4mm

# Densities in (kg/m3) 
rho_water = 1000
rho_temperedGlass =  2520
rho_PVB = 1120    

In [62]:
# MASS CALCULATIONS

V_water = pool_l * pool_w * pool_d                                                  # Volume of water in m3
m_water = round(V_water * rho_water,3)                                              #kg                          

m_temperedGlass = round( 2 * pool_l * pool_w * t_tempered * rho_temperedGlass, 3)   #kg
m_PVB = round(pool_l * pool_w * t_PVB * rho_PVB, 3)                                 #kg


In [64]:
# WEIGHT CALCULATIONS

W_water = m_water * g
W_temperedGlass = m_temperedGlass * g
W_PVB = m_PVB * g

W_glass = W_temperedGlass + W_PVB

W_total = W_water + W_glass


In [90]:
# CALCULATIONS FOR EACH FORCE ON TRUSS AND BEAMS

n = truss_num * beams_num

Feq = W_total * pool_l                       # Equivelent point load     

x_bar = pool_l/2                             # centre of truss

Ay = Feq/2
By = Feq/2
Ax = 0
Bx = 0

F_CD = (Feq - Ay)/(math.sin(convert_to_radians(40)))
F_CE = Ay + F_CD
F_BD = F_CD*math.cos(convert_to_radians(40)) + F_CE

F_AB = F_CD
F_BC = F_AB
F_AC = F_CE

In [174]:
# OUTPUT DATA

# Known Measurements
input_data = {
    "Known Measurements": [
        "Pool Length (m)", "Pool Width (m)", "Pool Depth (m)",
        "Glass Length (m)", "Glass Width (m)", "Glass Thickness (m)",
        "Density of Water (kg/m$^3$)", "Density of Tempered Glass (kg/m$^3$)", "Density of PVB (kg/m$^3$)"
    ],
    "Value": [
        pool_l, pool_w, pool_d,
        glass_slab_l, glass_slab_w, glass_slab_t,
        rho_water, rho_temperedGlass, rho_PVB,
    ]
}

# Calculated values
calculation_data = {
    "Calculated Forces": [
       "Water Weight", "Tempered Glass Weight", "PVB Weight", "Total Glass Weight", "Total Weight", "Feq"
    ],
    "Value": [
        W_water, W_temperedGlass, W_PVB, W_glass, W_total, Feq
    ], 
    "Approx": [
        W_water, W_temperedGlass, W_PVB, W_glass, W_total, Feq
    ]
}

# Force Analysis 
forces_data = {
    "Forces applied in System": [
        "Feq", "Ay", "By", "Ax", "Bx", "Segment CD", "Segment CE", "Segment BD", "Segment AB", "Segment BC", "Segment AC" 
    ],
    "Value": [
        Feq, Ay, By, Ax, Bx, F_CD, F_CE, F_BD, F_AB, F_BC, F_AC
    ], 
    "Approx": [
        Feq, Ay, By, Ax, Bx, F_CD, F_CE, F_BD, F_AB, F_BC, F_AC
    ]
}

# Create DataFrames
df_input = pd.DataFrame(input_data)
df_calculations = pd.DataFrame(calculation_data)
df_forces = pd.DataFrame(forces_data)

# apply formatting: 4 sig figs for Value and 3 sig figs for Approx
df_input["Value"] = df_input["Value"].map(lambda x: format_sig_fig(x, sig=4))

df_calculations["Value"] = df_calculations["Value"].map(
    lambda x: f"{format_sig_fig(x, sig=4)} N" if isinstance(x, (int, float)) else x
)

df_calculations["Approx"] = df_calculations["Approx"].map(
    lambda x: (
        f"{format_sig_fig(x / 1e6, sig=3)} MN" if isinstance(x, (int, float)) and x > 1e6 else
        f"{format_sig_fig(x / 1e3, sig=3)} kN" if isinstance(x, (int, float)) and x > 1e3 else
        f"{format_sig_fig(x, sig=3)} N"
    )
)

df_forces["Value"] = df_forces["Value"].map(
    lambda x: f"{format_sig_fig(x, sig=4)} N" if isinstance(x, (int, float)) else x
)

df_forces["Approx"] = df_forces["Approx"].map(
    lambda x: (
        f"{format_sig_fig(x / 1e6, sig=3)} MN" if isinstance(x, (int, float)) and x > 1e6 else
        f"{format_sig_fig(x / 1e3, sig=3)} kN" if isinstance(x, (int, float)) and x > 1e3 else
        f"{format_sig_fig(x, sig=3)} N"
    )
)


# Generate LaTeX for tables
latex_input = to_latex(df_input, "Known Measurements")
latex_calculations = to_latex(df_calculations, "Calculated Forces")
latex_forces = to_latex(df_forces, "Forces Applied in System")


# Generate LaTeX tables
latex_input = to_latex_bold_headers(
    df_input, 
    "The measured values of the glass pool and known values of densities.", 
    "known_measurements"
)
latex_calculations = to_latex_bold_headers(
    df_calculations, 
    "Calculated External Downward Forces on the truss system.", 
    "calculated_forces"
)
latex_forces = to_latex_bold_headers(
    df_forces, 
    "External and Internal forces applied in the truss system.", 
    "forces_applied_in_system"
)

# Print or save LaTeX tables
print(latex_input)
print(latex_calculations)
print(latex_forces)

# Save LaTeX tables to files
with open("input_table.tex", "w") as file:
    file.write(latex_input)

with open("calculations_table.tex", "w") as file:
    file.write(latex_calculations)

with open("forces_table.tex", "w") as file:
    file.write(latex_forces)
    

# Display DataFrames in Jupyter Notebook
display(df_input)
display(df_calculations)
display(df_forces)



\begin{table}
\caption{The measured values of the glass pool and known values of densities.}
\label{known_measurements}
\begin{tabular}{ll}
\toprule
\bfseries Known Measurements & \bfseries Value \\\midrule
\midrule
Pool Length (m) & 16.1 \\
Pool Width (m) & 16.1 \\
Pool Depth (m) & 0.24 \\
Glass Length (m) & 2.3 \\
Glass Width (m) & 1.15 \\
Glass Thickness (m) & 0.06 \\
Density of Water (kg/m$^3$) & 1000 \\
Density of Tempered Glass (kg/m$^3$) & 2520 \\
Density of PVB (kg/m$^3$) & 1120 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\caption{Calculated External Downward Forces on the truss system.}
\label{calculated_forces}
\begin{tabular}{lll}
\toprule
\bfseries Calculated Forces & \bfseries Value & \bfseries Approx \\\midrule
\midrule
Water Weight & 6.103e+05 N & 610 kN \\
Tempered Glass Weight & 3.768e+05 N & 377 kN \\
PVB Weight & 4.329e+04 N & 43.3 kN \\
Total Glass Weight & 4.201e+05 N & 420 kN \\
Total Weight & 1.0304e+06 N & 1.03 MN \\
Feq & 1.6589e+07 N & 16.6 MN \\
\

,Known Measurements,Value
0,Pool Length (m),16.1
1,Pool Width (m),16.1
2,Pool Depth (m),0.24
3,Glass Length (m),2.3
4,Glass Width (m),1.15
5,Glass Thickness (m),0.06
6,Density of Water (kg/m$^3$),1000
7,Density of Tempered Glass (kg/m$^3$),2520
8,Density of PVB (kg/m$^3$),1120


,Calculated Forces,Value,Approx
0,Water Weight,6.103e+05 N,610 kN
1,Tempered Glass Weight,3.768e+05 N,377 kN
2,PVB Weight,4.329e+04 N,43.3 kN
3,Total Glass Weight,4.201e+05 N,420 kN
4,Total Weight,1.0304e+06 N,1.03 MN
5,Feq,1.6589e+07 N,16.6 MN


,Forces applied in System,Value,Approx
0,Feq,1.6589e+07 N,16.6 MN
1,Ay,8.2944e+06 N,8.29 MN
2,By,8.2944e+06 N,8.29 MN
3,Ax,0 N,0 N
4,Bx,0 N,0 N
5,Segment CD,1.2904e+07 N,12.9 MN
6,Segment CE,2.1198e+07 N,21.2 MN
7,Segment BD,3.1083e+07 N,31.1 MN
8,Segment AB,1.2904e+07 N,12.9 MN
9,Segment BC,1.2904e+07 N,12.9 MN
